<a href="https://colab.research.google.com/github/eebjwa/SENSE_GEE_TEAM3/blob/main/CorineClassifier_WEEK4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#This will be our sentinel processing script
# Import earthengine API
import ee
# Authenticate and initialise 
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=CtkanJZWEN9v6ac_d4aANwbYmddpx28VoRooTH3lYDA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7qsGlkdGdezdDtzALzj59YA2egXjUILFplBJhWNjAWJyjTETKkdLI

Successfully saved authorization token.


In [9]:
def ESAcloudMask(img):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
     
    parsed by Nick Clinton
    """

    qa = img.select('QA60')

    # bits 10 and 11 are clouds and cirrus
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)

    # both flags set to zero indicates clear conditions.
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    # clouds is not clear
    cloud = clear.Not().rename(['ESA_clouds'])

    # return the masked and scaled data.
    return img.addBands(cloud)

In [12]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR")
S2coll = S2.filterBounds(ee.Geometry.Rectangle([28.0, 22.2,28.8,22.8])).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10).filterDate('2020-10-01','2020-11-01')
sample_image = S2coll.median()


In [14]:
import folium
#!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro

roi = ee.Geometry.Rectangle([28.0, 22.2,28.8,22.8])

map = folium.Map(location=[22.2,28.0],zoom_start=9)
map.addLayer(sample_image.clip(roi),{'min':0,'max':2000,'bands':['B4','B3','B2']},'rgb')
folium.LayerControl().add_to(map)
map

In [ ]:
dataset = ee.Image('COPERNICUS/CORINE/V20/100m/2018');
landCover = dataset.select('landcover');

class_vals = landCover.get('landcover_class_values').getInfo()
new_classes = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43]

In [16]:
#train a classifier on the corine europe data set using Sentinel 2
training_image = S2.filterBounds(ee.Geometry.Rectangle( -2.52298, 37.027956, -2.448138, 36.941432)).sort('CLOUDY_PIXEL_PERCENTAGE').first()
#train_roi = training_image.geometry()
train_roi = ee.Geometry.Rectangle( -2.52298, 37.027956, -2.448138, 36.941432)

bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

Corine_image = ee.Image("COPERNICUS/CORINE/V20/100m/2018").clip(train_roi)
print(Corine_image.getInfo())

map = folium.Map(location=[40.4,-3.7],zoom_start=9)
map.addLayer(sample_image.clip(roi),{'min':0,'max':2000,'bands':['B4','B3','B2']},'Target Image')
map.addLayer(training_image.clip(train_roi),{'min':0,'max':2000,'bands':['B4','B3','B2']},'Training Image')
map.addLayer(Corine_image.select('landcover').clip(train_roi), {}, 'landcover')
folium.LayerControl().add_to(map)
map

{'type': 'Image', 'bands': [{'id': 'landcover', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [81, 106], 'origin': [23016, 38578], 'crs': 'EPSG:3035', 'crs_transform': [100, 0, 900000, 0, -100, 5500000]}], 'version': 1578598451411424, 'id': 'COPERNICUS/CORINE/V20/100m/2018', 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-2.52298, 36.94143199999999], [-2.448138, 36.94143199999999], [-2.448138, 37.02795600000001], [-2.52298, 37.02795600000001], [-2.52298, 36.94143199999999]]]}, 'system:time_start': 1483228800000, 'landcover_class_names': ['Artificial surfaces; urban fabric; continuous urban fabric', 'Artificial surfaces; urban fabric; discontinuous urban fabric', 'Artificial surfaces; industrial, commercial, and transport units; industrial or commercial units', 'Artificial surfaces; industrial, commercial, and transport units; road and rail networks and associated land', 'Artificial surfaces; industrial, comm

In [17]:
Corine_coll = ee.ImageCollection("COPERNICUS/CORINE/V20/100m")
Pal = Corine_coll.select('landcover').get('landcover_class_palette')
Pal_names = Corine_coll.get('landcover_class_names')

In [18]:
dataset = ee.Image('COPERNICUS/CORINE/V20/100m/2018');
landCover = dataset.select('landcover');

class_vals = landCover.get('landcover_class_values').getInfo()
new_classes = [0,0,1,2,1,1,3,3,3,4,4,5,5,5,5,5,5,5,5,5,5,5,6,6,6,6,6,6,6,7,7,7,7,7,8,8,8,8,8,9,9,9,9,9]


dataset = ee.Image('COPERNICUS/CORINE/V20/100m/2018');
landCover = dataset.select('landcover');
print(landCover.get('landcover_class_palette').getInfo())
new_palette = ['E6004D','CC4DF2','000000','A600CC','80FF00','FFFF00','006600','A0A0A0','00FFFF','0000CC']

Corine_image_remapped = Corine_image.remap(class_vals, new_classes, 999, 'landcover')


map = folium.Map(location=[40.4,-3.7],zoom_start=9)
map.addLayer(Corine_image.select('landcover'), {}, 'landcover')
map.addLayer(Corine_image_remapped.select('remapped'),{'min':0, 'max':9, 'palette':new_palette},'remap')
folium.LayerControl().add_to(map)
map

['E6004D', 'FF0000', 'CC4DF2', 'CC0000', 'E6CCCC', 'E6CCE6', 'A600CC', 'A64DCC', 'FF4DFF', 'FFA6FF', 'FFE6FF', 'FFFFA8', 'FFFF00', 'E6E600', 'E68000', 'F2A64D', 'E6A600', 'E6E64D', 'FFE6A6', 'FFE64D', 'E6CC4D', 'F2CCA6', '80FF00', '00A600', '4DFF00', 'CCF24D', 'A6FF80', 'A6E64D', 'A6F200', 'E6E6E6', 'CCCCCC', 'CCFFCC', '000000', 'A6E6CC', 'A6A6FF', '4D4DFF', 'CCCCFF', 'E6E6FF', 'A6A6E6', '00CCF2', '80F2E6', '00FFA6', 'A6FFE6', 'E6F2FF']


In [19]:
#Corine_image_remapped = Corine_image.remap(class_vals, new_classes, 999, 'landcover')
print(Corine_image.getInfo())

combined_image = ee.ImageCollection([training_image.select(bands),Corine_image])

randomPoints = ee.FeatureCollection.randomPoints(train_roi,20000);
#print(randomPoints.getInfo())
label = 'remapped'

#
Corine_samples = Corine_image_remapped.sampleRegions(randomPoints,geometries=True)
S2_samples = training_image.select(bands).sampleRegions(Corine_samples)
#print(Corine_samples.getInfo())
#print(S2_samples.getInfo())
#print(Corine_samples.getInfo())

#training = training_image.select(bands).sampleRegions(Corine_samples,properties =[label],scale=100)

# Make a CART Classifier and train it 
#trained = ee.Classifier.smileCart().train(S2_samples, label, bands)
trained = ee.Classifier.minimumDistance().train(S2_samples, label, bands)

#Classify the input imagery 
classified = training_image.select(bands).classify(trained)

# Get a confusion matrix representing resubstitution accuracy.
trainAccuracy = trained.confusionMatrix()
#print('Resubstitution error matrix: ', trainAccuracy.getInfo())
#print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())


{'type': 'Image', 'bands': [{'id': 'landcover', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [81, 106], 'origin': [23016, 38578], 'crs': 'EPSG:3035', 'crs_transform': [100, 0, 900000, 0, -100, 5500000]}], 'version': 1578598451411424, 'id': 'COPERNICUS/CORINE/V20/100m/2018', 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-2.52298, 36.94143199999999], [-2.448138, 36.94143199999999], [-2.448138, 37.02795600000001], [-2.52298, 37.02795600000001], [-2.52298, 36.94143199999999]]]}, 'system:time_start': 1483228800000, 'landcover_class_names': ['Artificial surfaces; urban fabric; continuous urban fabric', 'Artificial surfaces; urban fabric; discontinuous urban fabric', 'Artificial surfaces; industrial, commercial, and transport units; industrial or commercial units', 'Artificial surfaces; industrial, commercial, and transport units; road and rail networks and associated land', 'Artificial surfaces; industrial, comm

In [ ]:
#Make pretty confusion matrix 
#import seaborn as sn
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt


#array = ValAccuracy.getInfo()
#df_cm = pd.DataFrame(array)
#plt.figure(figsize = (20,15))
#set color map 
#cmap = sn.cm.rocket_r
#matrix = sn.heatmap(df_cm, cmap=cmap)
#fig=matrix.get_figure()
#fig.savefig("error_matrix.png")


In [20]:
dataset = ee.Image('COPERNICUS/CORINE/V20/100m/2012');
landCover = dataset.select('landcover');

print(landCover.get('landcover_class_palette').getInfo())

['E6004D', 'FF0000', 'CC4DF2', 'CC0000', 'E6CCCC', 'E6CCE6', 'A600CC', 'A64DCC', 'FF4DFF', 'FFA6FF', 'FFE6FF', 'FFFFA8', 'FFFF00', 'E6E600', 'E68000', 'F2A64D', 'E6A600', 'E6E64D', 'FFE6A6', 'FFE64D', 'E6CC4D', 'F2CCA6', '80FF00', '00A600', '4DFF00', 'CCF24D', 'A6FF80', 'A6E64D', 'A6F200', 'E6E6E6', 'CCCCCC', 'CCFFCC', '000000', 'A6E6CC', 'A6A6FF', '4D4DFF', 'CCCCFF', 'E6E6FF', 'A6A6E6', '00CCF2', '80F2E6', '00FFA6', 'A6FFE6', 'E6F2FF']


In [21]:
landcover_vis = {'palette':['E6004D', 'FF0000', 'CC4DF2', 'CC0000', 'E6CCCC', 'E6CCE6', 'A600CC', 'A64DCC', 'FF4DFF', 'FFA6FF', 'FFE6FF', 'FFFFA8', 'FFFF00', 'E6E600', 'E68000', 'F2A64D', 'E6A600', 'E6E64D', 'FFE6A6', 'FFE64D', 'E6CC4D', 'F2CCA6', '80FF00', '00A600', '4DFF00', 'CCF24D', 'A6FF80', 'A6E64D', 'A6F200', 'E6E6E6', 'CCCCCC', 'CCFFCC', '000000', 'A6E6CC', 'A6A6FF', '4D4DFF', 'CCCCFF', 'E6E6FF', 'A6A6E6', '00CCF2', '80F2E6', '00FFA6', 'A6FFE6', 'E6F2FF']}
#class_vals = landCover.get('landcover_class_values').getInfo()
#new_classes = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43]

In [22]:
map = folium.Map(location=[40.4,-3.7],zoom_start=9)

#roi = ee.Geometry.Rectangle([10.5,52,12.5,53.5])

map.addLayer(training_image.clip(train_roi), {'min':0,'max':2000,'bands': ['B4', 'B3', 'B2']}, 'image')
map.addLayer(Corine_image_remapped.select('remapped'), {'min':0, 'max':9, 'palette':new_palette}, 'landcover',opacity = 0.6)
map.addLayer(randomPoints, {}, 'random points')
#map.addLayer(classified,landcover_vis,'classification')
folium.LayerControl().add_to(map)
map

In [ ]:
#Run classification over whole island 
Target_classified = sample_image.clip(roi).select(bands).classify(trained)

#Define individual ROIs for Haiti and DR 

#Clip classified image to each country (Haiti and DR)
#Haiti_classified = Hisp_classified.clip(Haiti_roi)
#DR_classified = Hisp_classified.clip(DR_roi)

new_palette = ['E6004D','CC4DF2','000000','A600CC','80FF00','FFFF00','006600','A0A0A0','00FFFF','0000CC']
simple_palette = ['FF0000','FF0000','000000','FF0000','80FF00','000000','80FF00','000000','000000','000000']

#sample_image.getInfo()
map = folium.Map(location=[40.4,-3.7],zoom_start=9)
map.addLayer(sample_image.clip(roi),{'min':0,'max':2000,'bands':['B4','B3','B2']},'rbg')
map.addLayer(sample_image.clip(roi),{'min':0,'max':2000,'bands':['B4']},'gray')
map.addLayer(Target_classified, {'min':0, 'max':9, 'palette':new_palette}, 'classified')
map.addLayer(Target_classified, {'min':0, 'max':9, 'palette':simple_palette}, 'classified - simple ')
#map.addLayer(DR_classified, {'min':0, 'max':43, 'palette':landcover_vis['palette']}, 'classified_DR')
folium.LayerControl().add_to(map)
map

In [ ]:
#Export images:
Export.image.toDrive({
  image: landsat,
  description: 'imageToCOGeoTiffExample',
  scale: 30,
  region: geometry,
  fileFormat: 'GeoTIFF',
  formatOptions: {
    cloudOptimized: true
  }
});

In [ ]:
#Validate the classifier against the europe data set

In [ ]:
# Get a confusion matrix representing resubstitution accuracy.
trainAccuracy = trained.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

Resubstitution error matrix:  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
#run the classifier on a single image on hispaniola to sense-check

In [ ]:
#create a collection of classified images from sentinel from 2016 to now

In [ ]:
#make pretty plots